# NEOM Potential Recharge Zones (region swale potential)

_Amgad Ellaboudy, James McCreight, Josh Daniel_  
_Eartshot Labs_

Mapping NEOM region for suitability of building features (such as swales) to capture surface/ sub-surface water, mainly to build up plant-available water in the soil.

Predictors:
1. Slope : Restricted to [0-10 degrees], gradual slopes are more desirable
2. Top Soil Types: Medium texture (loam/ silt) score highest, second is fine clays, and last is coarse sands for retaining plant-avaialble water
3. Runoff: Higher runoff is better, masked to values only above 1E-3 mm per grid box

## Preliminaries

In [1]:
import ee
import numpy as np

from earthshot import water_viz as vis
from earthshot import water_common as common
from earthshot import normalize as norm

from statistics import mean
import folium
from folium import plugins
from bokeh.plotting import figure, output_file, show
from bokeh.models import Range1d


Loading BokehJS ...

In [2]:
ee.Initialize()

bbox = ee.Geometry.BBox(34.52778, 27.72417, 35.96528, 29.14139)

bbox_name = 'neom'
#bbox = common.bboxes()[bbox_name]

## Predictors 
**Terrain Slope** from [Merit Hydro](https://developers.google.com/earth-engine/datasets/catalog/MERIT_Hydro_v1_0_1#description)

In [3]:
slope_img = ee.Image('users/jamesmcc/merit_slope/merit_terrain_slope').clip(bbox)
slope_mask = slope_img.lte(10).And(slope_img.gt(0))
slope_img_inv = slope_img.updateMask(slope_mask).pow(-1) 
slope_img_scaled = norm.img_scale(slope_img_inv, area_of_interest=bbox) 
norm.img_range(slope_img_scaled, area_of_interest=bbox)

[0, 1]

**Soil Types** qualitative ranking.

In [4]:
soil_types = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02").clip(bbox)

#categorizing soil types at 0, 10, and 30 cm depths based on retaining plant-available water ability
top_soils = [5,7,8,10]
medium_soils = [2,4,6,9]
low_soils = [1,3,11,12]
soil_0 = soil_types.expression(
        "(b('b0') == 5) ? 1.0" +
        ": (b('b0') == 7) ? 1.0" +
        ": (b('b0') == 8) ? 1.0" +
        ": (b('b0') == 10) ? 1.0" +
        ": (b('b0') == 2) ? 0.7" +
        ": (b('b0') == 4) ? 0.7" +
        ": (b('b0') == 6) ? 0.7" +
        ": (b('b0') == 9) ? 0.7" +
        ": (b('b0') == 1) ? 0.4" +
        ": (b('b0') == 3) ? 0.4" +
        ": (b('b0') == 11) ? 0.4" +
        ": (b('b0') == 12) ? 0.4" +
        ": 0")

soil_10 = soil_types.expression(
        "(b('b10') == 5) ? 1.0" +
        ": (b('b10') == 7) ? 1.0" +
        ": (b('b10') == 8) ? 1.0" +
        ": (b('b10') == 10) ? 1.0" +
        ": (b('b10') == 2) ? 0.7" +
        ": (b('b10') == 4) ? 0.7" +
        ": (b('b10') == 6) ? 0.7" +
        ": (b('b10') == 9) ? 0.7" +
        ": (b('b10') == 1) ? 0.4" +
        ": (b('b10') == 3) ? 0.4" +
        ": (b('b10') == 11) ? 0.4" +
        ": (b('b10') == 12) ? 0.4" +
        ": 0")

soil_30 = soil_types.expression(
        "(b('b30') == 5) ? 1.0" +
        ": (b('b30') == 7) ? 1.0" +
        ": (b('b30') == 8) ? 1.0" +
        ": (b('b30') == 10) ? 1.0" +
        ": (b('b30') == 2) ? 0.7" +
        ": (b('b30') == 4) ? 0.7" +
        ": (b('b30') == 6) ? 0.7" +
        ": (b('b30') == 9) ? 0.7" +
        ": (b('b30') == 1) ? 0.4" +
        ": (b('b30') == 3) ? 0.4" +
        ": (b('b30') == 11) ? 0.4" +
        ": (b('b30') == 12) ? 0.4" +
        ": 0")

top_soils = soil_0.expression('top_soil + soil_10 + soil_30',
                             {'top_soil': soil_0.select('constant'),
                             'soil_10': soil_10.select('constant'),
                             'soil_30': soil_30.select('constant')})

In [5]:
top_soils_scaled = norm.img_scale(top_soils, area_of_interest=bbox)
norm.img_range(top_soils_scaled, area_of_interest=bbox)

[0, 1]

**Runoff from [ERA5](https://code.earthengine.google.com/?asset=users/jamesmcc/era5_merit_potential_sfc_runoff_neom_climatology)**

In [6]:
runoff_clim = ee.ImageCollection('users/jamesmcc/era5_sfc_runoff_neom_climatology')
# convert from monthly m/hr to total mm/year for whole year
runoff_mm = runoff_clim.sum().clip(bbox).multiply(1000).multiply(365*24)

#convert from monthly m/hr to total mm/ season for each season
runoff_wint= (runoff_clim
           .filter(ee.Filter.inList('month', ee.List([12,1,2]))))
runoff_mm_wint= runoff_wint.sum().clip(bbox).multiply(1000).multiply(365*24)

runoff_spr= (runoff_clim
           .filter(ee.Filter.inList('month', ee.List([3,4,5]))))
runoff_mm_spr= runoff_spr.sum().clip(bbox).multiply(1000).multiply(365*24)

runoff_summer= (runoff_clim
               .filter(ee.Filter.inList('month', ee.List([6,7,8]))))
runoff_mm_summer= runoff_summer.sum().clip(bbox).multiply(1000).multiply(365*24)

runoff_fall= (runoff_clim
            .filter(ee.Filter.inList('month', ee.List([9,10,11]))))
runoff_mm_fall= runoff_fall.sum().clip(bbox).multiply(1000).multiply(365*24)

In [7]:
# Plot a histogram of the annual total runoff
runoff_hist = ee.Dictionary(
    runoff_mm
    .reduceRegion(
        ee.Reducer.histogram(), scale=100, bestEffort = True).get('surface_runoff'))

hist = np.array(runoff_hist.get('histogram').getInfo())
edges = (
    (np.array(list(range(len(hist) + 1))) * 
     np.array(runoff_hist.get('bucketWidth').getInfo())) + 
    np.array(runoff_hist.get('bucketMin').getInfo()) )

p = figure(
    title='NEOM Total Annual Runoff (mm)', 
    background_fill_color="white", height = 300)
_ = p.quad(
    top=hist,
    bottom=0,
    left=edges[:-1],
    right=edges[1:],
    fill_color="navy",
    line_color="navy",
    alpha=1.)
show(p)

In [8]:
#putting in threshold to mask annual runoff
runoff_threshold_mm = 20
runoff_mask = runoff_mm.gte(runoff_threshold_mm)
runoff_update = runoff_mm.updateMask(runoff_mask)
runoff_scaled = norm.img_scale(runoff_update, area_of_interest=bbox)

#putting a threshold and scaling seasonal runoff scores, 304.01 is maximum seasonal runoff (mm/season) in winter

runoff_threshold_mm_seas = 5
runoff_mask_wint = runoff_mm_wint.gte(runoff_threshold_mm_seas)
runoff_update_wint = runoff_mm_wint.updateMask(runoff_mask_wint)
runoff_scaled_wint = norm.img_scale(runoff_update_wint, max = 304.01, area_of_interest=bbox)
norm.img_range(runoff_scaled_wint, area_of_interest=bbox)

runoff_mask_spr = runoff_mm_spr.gte(runoff_threshold_mm_seas)
runoff_update_spr = runoff_mm_spr.updateMask(runoff_mask_spr)
runoff_scaled_spr = norm.img_scale(runoff_update_spr, max = 304.01, area_of_interest=bbox)
norm.img_range(runoff_scaled_spr, area_of_interest=bbox)

runoff_mask_summer = runoff_mm_summer.gte(runoff_threshold_mm_seas)
runoff_update_summer = runoff_mm_summer.updateMask(runoff_mask_summer)
runoff_scaled_summer = norm.img_scale(runoff_update_summer, max = 304.01, area_of_interest=bbox)
norm.img_range(runoff_scaled_summer, area_of_interest=bbox)

runoff_mask_fall = runoff_mm_fall.gte(runoff_threshold_mm_seas)
runoff_update_fall = runoff_mm_fall.updateMask(runoff_mask_fall)
runoff_scaled_fall = norm.img_scale(runoff_update_fall, max = 304.01, area_of_interest=bbox)
norm.img_range(runoff_scaled_fall, area_of_interest=bbox)

[0, 0.28459507230621983]

## Calculate the PRZ score

In [9]:
score = (runoff_scaled.multiply(.5)
         .add(top_soils_scaled.multiply(.25))
         .add(slope_img_scaled.multiply(.25)))
score = norm.img_scale(score, area_of_interest=bbox).rename('score')
norm.img_range(score, area_of_interest=bbox)

#calculate seasonal scores as well, 0.711 is maximum winter PRZ score
score_wint = (runoff_scaled_wint.multiply(.5)
         .add(top_soils_scaled.multiply(.25))
         .add(slope_img_scaled.multiply(.25)))
score_wint = norm.img_scale(score_wint, area_of_interest=bbox).rename('score_wint')

score_spr = (runoff_scaled_spr.multiply(.5)
         .add(top_soils_scaled.multiply(.25))
         .add(slope_img_scaled.multiply(.25)))
score_spr = norm.img_scale(score_spr, max = 0.711, area_of_interest=bbox).rename('score_spr')

score_summer = (runoff_scaled_summer.multiply(.5)
         .add(top_soils_scaled.multiply(.25))
         .add(slope_img_scaled.multiply(.25)))
score_summer = norm.img_scale(score_summer, max = 0.711, area_of_interest=bbox).rename('score_summer')

score_fall = (runoff_scaled_fall.multiply(.5)
         .add(top_soils_scaled.multiply(.25))
         .add(slope_img_scaled.multiply(.25)))
score_fall = norm.img_scale(score_fall, max = 0.711, area_of_interest=bbox).rename('score_fall')

In [10]:
# Plot a histogram of the scores
score_hist = ee.Dictionary(
    score.clip(bbox)
    .reduceRegion(
        ee.Reducer.histogram()).get('score'))

hist = np.array(score_hist.get('histogram').getInfo())
edges = (
    (np.array(list(range(len(hist) + 1))) * 
     np.array(score_hist.get('bucketWidth').getInfo())) + 
    np.array(score_hist.get('bucketMin').getInfo()) )

p = figure(
    title='NEOM PRZ Score', 
    background_fill_color="white", height = 300)
_ = p.quad(
    top=hist,
    bottom=0,
    left=edges[:-1],
    right=edges[1:],
    fill_color="navy",
    line_color="navy",
    alpha=1.)
show(p)

In [11]:
# Map parameters
box_corners = bbox.toGeoJSON()['coordinates'][0]
center_lon = mean([corner[0] for corner in box_corners])
center_lat = mean([corner[1] for corner in box_corners])

palette_name = 'Blues'
palette_len = 5

vis_min = 0
vis_max = 1

palette = vis.brewer[palette_name][palette_len][::-1]
vis.legend(
    palette=palette, minimum=vis_min, maximum=vis_max, title='NEOM PRZ Score')

vis_params = {
    'min': vis_min, 'max': vis_max, 'dimensions': 512,
    'palette': palette}

In [12]:
# plotting score for NEOM region
the_map = vis.folium_map(location=[center_lat-2.5, center_lon], zoom_start=8, height=1500)
the_map.add_ee_layer(score, vis_params, name = 'NEOM PRZ Score')
the_map.add_ee_layer(score_wint, vis_params, name = 'NEOM PRZ winter Score')
the_map.add_ee_layer(score_spr, vis_params, name = 'NEOM PRZ spring Score')
the_map.add_ee_layer(score_summer, vis_params, name = 'NEOM PRZ summer Score')
the_map.add_ee_layer(score_fall, vis_params, name = 'NEOM PRZ fall Score')
the_map.add_ee_layer(runoff_scaled, vis_params, name = 'Runoff')
vis.folium_display(the_map)

## Export map

In [13]:
collection_name = ('users/jamesmcc/NEOM')
max_pixels = 100000000000  #For global is quite large
description = "NEOM PRZ Score"

In [14]:
# neom_asset = ee.data.createAsset(
#     {'type': 'ImageCollection'}, collection_name)
# _ = ee.batch.Export.image.toAsset(
#         score,
#         description=description, 
#         assetId=neom_asset['id'] + '/' + 'NEOM_PRZ_Score',
#         region=bbox, 
#         scale=100,
#         maxPixels=max_pixels).start()

## Potential increases in recharge

In [15]:
#calculate sum of potential water storage from neom area over a year
def sum_runoff(runoff_img, geometry, scale):
    return (
        runoff_img
        .reduceRegion(reducer= ee.Reducer.sum(), geometry=geometry, scale=100)
        .getInfo()['surface_runoff']*(scale*scale))

def sum_runoff_score_thresh(runoff_img, score, thresh, geometry, scale):
  thresh_mask = score.gte(thresh)
  return sum_runoff(runoff_img.updateMask(thresh_mask), geometry, scale)

In [16]:
# cubic m / year
potential_recharge = sum_runoff(runoff_mm.divide(1000), bbox, 100)
thresh_recharge = [
    sum_runoff_score_thresh(runoff_mm.divide(1000), score, thresh, bbox, 100) 
    for thresh in [0, .2, .4, .6, .8]]

In [17]:
recharge_indiv = np.diff(thresh_recharge[::-1]).tolist()[::-1] + [thresh_recharge[-1]]

In [18]:
names = ['all runoff', 'all PRZ zones', '0.0 - 0.2', '0.2 - 0.4', '0.4 - 0.6', '0.6 - 0.8', '0.8 - 1.0']
values = [potential_recharge] + [sum(recharge_indiv)] + recharge_indiv

In [19]:
from bokeh.io import output_file, show
from bokeh.plotting import figure
p = figure(x_range=names, plot_height=250, 
           title="NEOM Potential PRZ Recharge By Score Range (mm^3/yr)",
           toolbar_location=None, tools="")
p.vbar(x=names, top=values, width=0.9)
p.xgrid.grid_line_color = None
p.y_range.start = 1
show(p)

This makes sense given the histogram of the score, where most of the points/density is in [0.2, 0.4]. If we stratify by percentiles, we get a better sense of how much recharge potential resides in groups of scores (by percentiles).  

In [20]:
# do it by percentiles
score_percentiles = ee.Dictionary(
    score.clip(bbox)
    .reduceRegion(
        ee.Reducer.percentile([0, 20, 40, 60, 80])))

In [21]:
percentiles = sorted(score_percentiles.values().getInfo())

In [22]:
percentile_recharge = [
    sum_runoff_score_thresh(runoff_mm.divide(1000), score, thresh, bbox, 100) 
    for thresh in percentiles]

In [23]:
pct_recharge_indiv = np.diff(percentile_recharge[::-1]).tolist()[::-1] + [percentile_recharge[-1]]

In [24]:
names = ['all runoff', 'all swale zones', '0.0 - 0.2', '0.2 - 0.4', '0.4 - 0.6', '0.6 - 0.8', '0.8 - 1.0']
values = [potential_recharge] + [sum(recharge_indiv)] + pct_recharge_indiv

In [25]:
from bokeh.io import output_file, show
from bokeh.plotting import figure
p = figure(x_range=names, plot_height=250, 
           title="NEOM Potential Swale Recharge By Percentile Range (m^3/yr)",
           toolbar_location=None, tools="")
p.vbar(x=names, top=values, width=0.9)
p.xgrid.grid_line_color = None
p.y_range.start = 1
show(p)

Because the score preferentially weights runoff availability, this score makes sense that the higher range of percentiles, the higher the recharge. 

In [26]:
## Can mask out streams as a further step.